Already exists:
- Boligrafica Twitter Tweepy
- Greenhouse Summon
- NOAA Data update GHD

Want to create:
- GFW - Datos Bolivia Deforestacion / Incendios
- Other twitter bots!??

Types of conditions:
- Runs on a schedule 
  - once every week (like boligrafica)
  - 4 times a day (like greenhouse summon)
- Runs if it has detected updated data (if new data, run)
  - Like NOAA data in GHD

In [1]:
import os
import sys
import pandas as pd
import datetime
from IPython import get_ipython

# environmental secrets when working locally
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
def NEW_DATA_RUNNER():
    
    return

In [3]:
def SCHEDULED_RUNNER(notebook_path, schedule_in_days = None):
    
    print("starting: ", notebook_path)
    
    if schedule_in_days == None:
        EXECUTE = True
        
    else:
        # check log
        log = pd.read_csv("log.csv", index_col=0)
        try:
            last_executed = log.loc[notebook_path]["last_executed"]
            print(f"previous log found: {last_executed}")
        except:
            print("no log found (first time?) - creating dummy entry (year 2000)")
            log.loc[notebook_path] = "2002-07-16 18:32"
            
            # try again
            last_executed = log.loc[notebook_path]["last_executed"]
            
        # get time difference in days
        now = datetime.datetime.now()
        last_executed = datetime.datetime.strptime(last_executed, '%Y-%m-%d %H:%M')
        diff_seconds = now - last_executed
        diff_days = diff_seconds.total_seconds() / 60 /60 /24
        print(f"difference in execution is: {round(diff_days,4)} days")
        print(f"needed difference to execute: {round(schedule_in_days,4)} days")
        
        # FINAL CHECK
        if diff_days > schedule_in_days:
            EXECUTE = True
        else:
            EXECUTE = False
            

        
    print("Execute is: ", str(EXECUTE))
    if EXECUTE == True:

        
        # go to folder to run script locally
        folder_path = notebook_path.split("/")[:-1]
        os.chdir("/".join(folder_path))

        try:
            get_ipython().run_line_magic("run", notebook_path.split("/")[-1])
            status = "SUCCESS!"
            print(status)
        except:
            status = "FAILURE."
            print(status)

        # go back to root repository
        os.chdir("".join(["../"] * len(folder_path)))
        
        
        # log
        log.loc[notebook_path] = [datetime.datetime.now().strftime("%Y-%m-%d %H:%M"), status]
        log.to_csv("log.csv")
        print("New log entry successful!\n\n")
    else:
        print("Did not execute nor log\n\n")
    return

########################################################
#

SCHEDULED_RUNNER(notebook_path = "Greenhouse_Data/Twitter/GreenhouseSummon.ipynb", schedule_in_days= 6/24)    
SCHEDULED_RUNNER(notebook_path = "Greenhouse_Data/Random_Fig_Daily/Random_Fig_Daily.ipynb", schedule_in_days= 1)    
SCHEDULED_RUNNER(notebook_path = "Greenhouse_Data/NOAA_Updates/NOAA_Update.ipynb", schedule_in_days= 1)


starting:  Greenhouse_Data/Twitter/GreenhouseSummon.ipynb
previous log found: 2022-07-28 16:14
difference in execution is: 0.2902 days
needed difference to execute: 0.25 days
Execute is:  True
Iran, Islamic Republic of greenhouse gas emission data and figures 👇

If you like this kind of stuff be sure to check out my website for many more datasets and figures! Thank you and have a wonderful day 🤖
Starting success list []
 > Searching for 20 tweets containing "greenhouse gas emissions Trinidad and Tobago"
 > found 0 tweets
 > empty df: NEXT ITERATION

Starting success list []
 > Searching for 20 tweets containing "greenhouse gas emissions Eritrea"
 > found 0 tweets
 > empty df: NEXT ITERATION

Starting success list []
 > Searching for 20 tweets containing "greenhouse gas emissions Iraq"
 > found 0 tweets
 > empty df: NEXT ITERATION

Starting success list []
 > Searching for 20 tweets containing "greenhouse gas emissions Montserrat"
 > found 0 tweets
 > empty df: NEXT ITERATION

Starting 

['country_data/ZAF_South Africa/figures/ZAF_relative_totals.png',
 'country_data/ZAF_South Africa/figures/ZAF_GCP_Country_Highlight.png',
 'country_data/ZAF_South Africa/figures/ZAF_GCP_1.png',
 'country_data/ZAF_South Africa/figures/ZAF_Minx_top20_subsectors.png']

 > saved and selected 4 images

 > OH NO! NO TWEETS WERE FOUND NOR TWEETED
success list: [False] 



Starting success list [False]
 > Searching for 20 tweets containing "greenhouse gas emissions Mauritius"
 > found 0 tweets
 > empty df: NEXT ITERATION

Starting success list [False]
 > Searching for 20 tweets containing "greenhouse gas emissions China"
 > found 20 tweets
 > days of invalidity: 20
 > Number of invalids: 76
 > 0 usernames filtered out
 > retweets removed: 7


['country_data/CHN_China/figures/CHN_relative_totals.png',
 'country_data/CHN_China/figures/CHN_GCP_Country_Highlight.png',
 'country_data/CHN_China/figures/CHN_GCP_1.png',
 'country_data/CHN_China/figures/CHN_Minx_top20_subsectors.png']

 > saved and selected 4 images
 > TWEET SUCCESSFUL!!
success list: [False, True] 



Starting success list [False, True]
 > Searching for 20 tweets containing "greenhouse gas emissions Portugal"
 > found 0 tweets
 > empty df: NEXT ITERATION

Starting success list [False, True]
 > Searching for 20 tweets containing "greenhouse gas emissions Turkey"
 > found 0 tweets
 > empty df: NEXT ITERATION

Starting success list [False, True]
 > Searching for 20 tweets containing "greenhouse gas emissions Jamaica"
 > found 0 tweets
 > empty df: NEXT ITERATION

Starting success list [False, True]
 > Searching for 20 tweets containing "greenhouse gas emissions Mexico"
 > found 3 tweets
 > days of invalidity: 20
 > Number of invalids: 77
 > 0 usernames filtered out
 > retweets removed: 2


['country_data/MEX_Mexico/figures/MEX_relative_totals.png',
 'country_data/MEX_Mexico/figures/MEX_GCP_Country_Highlight.png',
 'country_data/MEX_Mexico/figures/MEX_GCP_1.png',
 'country_data/MEX_Mexico/figures/MEX_Minx_top20_subsectors.png']

 > saved and selected 4 images
 > TWEET SUCCESSFUL!!
success list: [False, True, True] 





 FINISHED WHILE LOOP


 FINISHED WHILE LOOP
TOOO SOON!!
SUCCESS!
New log entry successful!


starting:  Greenhouse_Data/Random_Fig_Daily/Random_Fig_Daily.ipynb
previous log found: 2022-07-28 16:14
difference in execution is: 0.2906 days
needed difference to execute: 1 days
Execute is:  False
Did not execute nor log


starting:  Greenhouse_Data/NOAA_Updates/NOAA_Update.ipynb
previous log found: 2022-07-28 16:15
difference in execution is: 0.2899 days
needed difference to execute: 1 days
Execute is:  False
Did not execute nor log


